In [45]:
import tensorflow as tf
import numpy as np
import cv2
import input_data

In [46]:
img_height=480
img_width=360
#Network Parameters
n_input=img_height*img_width*3
learning_rate=1e-8
training_iters=500
batch_size=2
display_step=1
dropout=0.8
epoch=2

#tf graph input
x=tf.placeholder(tf.float32,[None,img_height,img_width,3])
y=tf.placeholder(tf.float32,[None,img_height,img_width,16])
keep_prob=tf.placeholder(tf.float32)

In [47]:
def conv2d(img,w,b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(img,w,strides=[1,1,1,1],padding='SAME'),b))
def max_pool(img,k):
    return tf.nn.max_pool(img,ksize=[1,k,k,1],strides=[1,k,k,1],padding='SAME')
def conv2d_transpose(img,w,b,output_shape):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d_transpose(img,w,output_shape=output_shape,strides=[1,1,1,1],padding='SAME'),b))
def cv_bottleneck(img,w1,w2):
    tmp_bn_1=tf.nn.relu(tf.nn.conv2d(img,w1,strides=[1,1,1,1],padding='SAME'))
    tmp_bn_2=tf.nn.conv2d(tmp_bn_1,w2,strides=[1,1,1,1],padding='SAME')
    return (img+tmp_bn_2)
def dc_bottleneck(img,w1,w2,output_shape):
    output_shape[3]=w1.get_shape().as_list()[2]
    tmp_bn_1=tf.nn.relu(tf.nn.conv2d_transpose(img,w1,output_shape=output_shape,strides=[1,1,1,1],padding='SAME'))
    output_shape[3]=w2.get_shape().as_list()[2]
    tmp_bn_2=tf.nn.conv2d_transpose(tmp_bn_1,w2,output_shape=output_shape,strides=[1,1,1,1],padding='SAME')
    return (img+tmp_bn_2)
def hourglass(img,wc1_1,wc1_2,bc1,wc2_1,wc2_2,bc2,wc3_1,wc3_2,bc3,wc4_1,wc4_2,bc4,wc5_1,wc5_2,bc5):
    tmp_cv_1=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(img,wc1_1,wc1_2),bc1))
    tmp_cv_1_pooling=max_pool(tmp_cv_1,k=2)
    tmp_cv_1_pooling=tf.nn.dropout(tmp_cv_1_pooling,keep_prob)
    
    tmp_cv_2=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_cv_1_pooling,wc2_1,wc2_2),bc2))
    tmp_cv_2_pooling=max_pool(tmp_cv_2,k=2)
    tmp_cv_2_pooling=tf.nn.dropout(tmp_cv_2_pooling,keep_prob)
    
    tmp_cv_3=tf.nn.relu(tf.nn.bias_add(cv_bottleneck(tmp_cv_2_pooling,wc3_1,wc3_2),bc3))
    tmp_cv_3=tf.nn.dropout(tmp_cv_3,keep_prob)
    
    tmp_rs_1=tf.image.resize_bilinear(tmp_cv_3,tmp_cv_2.get_shape().as_list()[1:3])
    tmpShape=tmp_cv_1_pooling.get_shape().as_list()
    tmpShape[0]=batch_size
    tmp_dc_1=tf.nn.relu(tf.nn.bias_add(dc_bottleneck(tmp_rs_1,wc4_1,wc4_2,output_shape=tmpShape),bc4))
    
    tmp_add_1=tf.add(tmp_dc_1,tmp_cv_1_pooling)
    tmp_add_1=tf.nn.dropout(tmp_add_1,keep_prob)
    
    tmp_rs_2=tf.image.resize_bilinear(tmp_add_1,tmp_cv_1.get_shape().as_list()[1:3])
    tmpShape=img.get_shape().as_list()
    tmpShape[0]=batch_size
    tmp_dc_2=tf.nn.relu(tf.nn.bias_add(dc_bottleneck(tmp_rs_2,wc5_1,wc5_2,output_shape=tmpShape),bc5))
    
    tmp_add_2=tf.add(tmp_dc_2,img)
    tmp_add_21=tf.nn.dropout(tmp_add_2,keep_prob)
    
    return (tmp_add_2)

In [48]:
wc1=tf.Variable(tf.random_normal([7,7,3,16]),name="wc1")
bc1=tf.Variable(tf.random_normal([16]),name="bc1")

wh1_1_1=tf.Variable(tf.random_normal([5,5,16,64]),name="wh1_1_1")
wh1_1_2=tf.Variable(tf.random_normal([5,5,64,16]),name="wh1_1_2")

bh1_1=tf.Variable(tf.random_normal([16]),name="bh1_1")

wh1_2_1=tf.Variable(tf.random_normal([3,3,16,64]),name="wh1_2_1")
wh1_2_2=tf.Variable(tf.random_normal([3,3,64,16]),name="wh1_2_2")

bh1_2=tf.Variable(tf.random_normal([16]),name="bh1_2")

wh1_3_1=tf.Variable(tf.random_normal([3,3,16,16]),name="wh1_3_1")
wh1_3_2=tf.Variable(tf.random_normal([3,3,16,16]),name="wh1_3_2")

bh1_3=tf.Variable(tf.random_normal([16]),name="bh1_3")

wh1_4_1=tf.Variable(tf.random_normal([3,3,64,16]),name="wh1_4_1")
wh1_4_2=tf.Variable(tf.random_normal([3,3,16,64]),name="wh1_4_2")

bh1_4=tf.Variable(tf.random_normal([16]),name="bh1_4")

wh1_5_1=tf.Variable(tf.random_normal([3,3,64,16]),name="wh1_5_1")
wh1_5_2=tf.Variable(tf.random_normal([3,3,16,64]),name="wh1_5_2")

bh1_5=tf.Variable(tf.random_normal([16]),name="bh1_5")

wd1=tf.Variable(tf.random_normal([7,7,16,16]),name="wd1")
bd1=tf.Variable(tf.random_normal([16]),name="bd1")

wc2=tf.Variable(tf.random_normal([1,1,16,16]),name="wc2")
bc2=tf.Variable(tf.random_normal([16]),name="bc2")

In [49]:
#Construct model
conv1=conv2d(x,wc1,bc1)
conv1_pooling=max_pool(conv1,k=2)
conv1_pooling=tf.nn.dropout(conv1_pooling,keep_prob)

hg_1=hourglass(conv1_pooling,wh1_1_1,wh1_1_2,bh1_1,wh1_2_1,wh1_2_2,bh1_2,wh1_3_1,wh1_3_2,bh1_3,wh1_4_1,wh1_4_2,bh1_4,wh1_5_1,wh1_5_2,bh1_5)

resize1=tf.image.resize_bilinear(hg_1,conv1.get_shape().as_list()[1:3])
tmpShape=x.get_shape().as_list()
tmpShape[0]=batch_size
tmpShape[3]=wd1.get_shape().as_list()[2]
dconv1=conv2d_transpose(resize1,wd1,bd1,output_shape=tmpShape)
dconv1=tf.nn.dropout(dconv1,keep_prob)

conv2=conv2d(dconv1,wc2,bc2)
conv2=tf.nn.dropout(conv2,keep_prob)

cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=conv2,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [50]:
saver = tf.train.Saver({"wc1":wc1,"bc1":bc1,"wh1_1_1":wh1_1_1,"wh1_1_2":wh1_1_2,"bh1_1":bh1_1,"wh1_2_1":wh1_2_1,"wh1_2_2":wh1_2_2,
                        "bh1_2":bh1_2,"wh1_3_1":wh1_3_1,"wh1_3_2":wh1_3_2,"bh1_3":bh1_3,"wh1_4_1":wh1_4_1,"wh1_4_2":wh1_4_2,
                        "bh1_4":bh1_4,"wh1_5_1":wh1_5_1,"wh1_5_2":wh1_5_2,"bh1_5":bh1_5,
                        "wd1":wd1,"bd1":bd1,"wc2":wc2,"bc2":bc2
                       })
init=tf.global_variables_initializer()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

In [51]:
with tf.Session(config=config) as sess:
    sess.run(init)
    saver = tf.train.Saver()
    #saver.restore(sess,"model/model.ckpt")
    for i in range(0,epoch):
        print("epoch:"+str(i)+"       !!!!!!!!!!!!")
        step=0
        now_at=0
        while step*batch_size<training_iters:
            X=input_data.x_next_batch(
                    img_dir_path='mpii_human_pose_v1\\output_images360x480\\',
                    index_path='train_data\\new_data.json',
                    img_height=img_height,img_width=img_width,
                    batch_size=batch_size,now_at=now_at)
            Y=input_data.y_next_batch(
                    img_dir_path='mpii_human_pose_v1\\output_images360x480\\',
                    index_path='train_data\\new_data.json',
                    img_height=img_height,img_width=img_width,
                    batch_size=batch_size,now_at=now_at)
            sess.run(optimizer,feed_dict = {x:X,y:Y,keep_prob:dropout})
            loss=sess.run(cost,feed_dict = {x:X,y:Y,keep_prob:1.})
            if step%display_step==0:
                print("Iter "+str(step*batch_size)+", Minibatch Loss="+"{:.6f}".format(loss))
            step+=1
            now_at+=batch_size
        save_path = saver.save(sess, "model/model_hg.ckpt")
print("done!")

epoch:0       !!!!!!!!!!!!
Iter 0, Minibatch Loss=819129116262400.000000
Iter 2, Minibatch Loss=1284763499364352.000000
Iter 4, Minibatch Loss=819087508766720.000000
Iter 6, Minibatch Loss=549176496619520.000000
Iter 8, Minibatch Loss=1728315241005056.000000
Iter 10, Minibatch Loss=988739790700544.000000
Iter 12, Minibatch Loss=291007354634240.000000
Iter 14, Minibatch Loss=322687335399424.000000
Iter 16, Minibatch Loss=872987200847872.000000
Iter 18, Minibatch Loss=1102678025306112.000000
Iter 20, Minibatch Loss=672247744299008.000000
Iter 22, Minibatch Loss=1013015885381632.000000
Iter 24, Minibatch Loss=887994688995328.000000
Iter 26, Minibatch Loss=379253296201728.000000
Iter 28, Minibatch Loss=506563039068160.000000
Iter 30, Minibatch Loss=525926697795584.000000
Iter 32, Minibatch Loss=371327303155712.000000
Iter 34, Minibatch Loss=414719995281408.000000
Iter 36, Minibatch Loss=1276310601072640.000000
Iter 38, Minibatch Loss=1453826297036800.000000
Iter 40, Minibatch Loss=15737562

Iter 344, Minibatch Loss=823542329376768.000000
Iter 346, Minibatch Loss=800133616762880.000000
Iter 348, Minibatch Loss=1232086530785280.000000
Iter 350, Minibatch Loss=1239011091808256.000000
Iter 352, Minibatch Loss=1081340359344128.000000
Iter 354, Minibatch Loss=584771474292736.000000
Iter 356, Minibatch Loss=402963361169408.000000
Iter 358, Minibatch Loss=316046376435712.000000
Iter 360, Minibatch Loss=416821408694272.000000
Iter 362, Minibatch Loss=414963667566592.000000
Iter 364, Minibatch Loss=310490802683904.000000
Iter 366, Minibatch Loss=256541634592768.000000
Iter 368, Minibatch Loss=389223257473024.000000
Iter 370, Minibatch Loss=795804759490560.000000
Iter 372, Minibatch Loss=108604564701184.000000
Iter 374, Minibatch Loss=75677801906176.000000
Iter 376, Minibatch Loss=78092554993664.000000
Iter 378, Minibatch Loss=650861155975168.000000
Iter 380, Minibatch Loss=596161660452864.000000
Iter 382, Minibatch Loss=1194149889966080.000000
Iter 384, Minibatch Loss=3832130211676

Iter 186, Minibatch Loss=1410396258828288.000000
Iter 188, Minibatch Loss=596817918033920.000000
Iter 190, Minibatch Loss=2068264184184832.000000
Iter 192, Minibatch Loss=2040548626006016.000000
Iter 194, Minibatch Loss=91892544962560.000000
Iter 196, Minibatch Loss=83398584434688.000000
Iter 198, Minibatch Loss=54387221200896.000000
Iter 200, Minibatch Loss=57366041067520.000000
Iter 202, Minibatch Loss=77869510295552.000000
Iter 204, Minibatch Loss=85176918999040.000000
Iter 206, Minibatch Loss=64222486593536.000000
Iter 208, Minibatch Loss=38142669225984.000000
Iter 210, Minibatch Loss=104379768111104.000000
Iter 212, Minibatch Loss=77384741027840.000000
Iter 214, Minibatch Loss=80024283643904.000000
Iter 216, Minibatch Loss=62466251816960.000000
Iter 218, Minibatch Loss=87536651206656.000000
Iter 220, Minibatch Loss=74353098096640.000000
Iter 222, Minibatch Loss=45396109819904.000000
Iter 224, Minibatch Loss=449333304492032.000000
Iter 226, Minibatch Loss=500729164857344.000000
Ite